# Barren Plateaus in Parameterized Quantum Circuits

Barren plateaus are one of the biggest challenges for scaling variational quantum algorithms. When the gradient of the cost function vanishes exponentially with the number of qubits, the optimization landscape becomes essentially flat — and no classical optimizer can find a good direction.

In this notebook, we empirically investigate:
1. How gradient variance scales with circuit width (number of qubits)
2. How circuit depth affects trainability
3. Whether global vs local cost functions make a difference
4. Strategies for avoiding barren plateaus: initialization, layer-wise training

We use the **parameter shift rule** to compute exact analytical gradients, averaged over many random initializations to estimate the gradient variance.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.circuits import (
    build_hardware_efficient_ansatz, build_structured_ansatz,
    build_identity_block_ansatz, get_ansatz_builders
)
from src.gradients import (
    compute_gradient_parameter_shift, compute_gradient_variance,
    sweep_gradient_variance, sweep_depth_variance
)
from src.cost_functions import (
    global_cost_observable, local_cost_observable,
    two_local_cost_observable
)
from src.trainability import (
    layerwise_train, random_init, correlated_init, identity_init
)
from src.plotting import (
    plot_gradient_variance_vs_qubits, plot_gradient_variance_vs_depth,
    plot_global_vs_local_cost, plot_cost_landscape_1d,
    plot_variance_heatmap, plot_layerwise_convergence,
    plot_ansatz_comparison
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True)

print('Setup complete!')

## 1. The Parameter Shift Rule

Before we measure gradients, let's understand how. The **parameter shift rule** gives us exact analytical gradients for parameterized quantum circuits:

$$\frac{\partial \langle C \rangle}{\partial \theta_k} = \frac{1}{2}\left[\langle C \rangle_{\theta_k + \pi/2} - \langle C \rangle_{\theta_k - \pi/2}\right]$$

We evaluate the circuit at two shifted parameter values and take the difference. No finite-difference approximation needed.

Let's verify this works on a small circuit.

In [ ]:
# quick sanity check: gradient of a simple circuit
circuit = build_hardware_efficient_ansatz(n_qubits=2, depth=1)
observable = global_cost_observable(n_qubits=2)

print(f"Circuit has {circuit.num_parameters} parameters")
print(f"Circuit depth: {circuit.depth()}")
print()

# compute gradient at a random point
rng = np.random.default_rng(42)
params = rng.uniform(0, 2 * np.pi, circuit.num_parameters)

print("Gradients at a random parameter point:")
for i in range(min(4, circuit.num_parameters)):
    grad = compute_gradient_parameter_shift(
        circuit, observable, params, param_index=i
    )
    print(f"  d<C>/d(theta_{i}) = {grad:.6f}")

## 2. Experiment 1: Gradient Variance vs Number of Qubits

This is the core barren plateau experiment. We measure how the variance of the gradient changes as we add more qubits to the circuit.

**Theory predicts**: For a hardware-efficient ansatz with a global cost function, the gradient variance should decay *exponentially* with the number of qubits:

$$\text{Var}\left[\frac{\partial C}{\partial \theta_k}\right] \sim \mathcal{O}\left(\frac{1}{2^n}\right)$$

This means a 10-qubit circuit has ~1000x smaller gradients than a 2-qubit circuit. Let's see if this holds.

In [ ]:
# sweep qubit count with hardware-efficient ansatz + global cost
qubit_range = [2, 4, 6, 8, 10, 12]
depth = 4  # fixed depth
n_samples = 200  # random initializations per configuration

print(f"Sweeping qubits {qubit_range} with depth={depth}, {n_samples} samples each")
print("This takes a few minutes...\n")

hw_global_results = sweep_gradient_variance(
    qubit_range=qubit_range,
    depth=depth,
    ansatz_builder=build_hardware_efficient_ansatz,
    observable_builder=global_cost_observable,
    n_samples=n_samples
)

print("\nResults:")
for n, var in zip(qubit_range, hw_global_results['variances']):
    print(f"  {n} qubits: variance = {var:.6e}")

In [ ]:
# plot on log scale
fig, ax = plt.subplots(figsize=(10, 6))

variances = hw_global_results['variances']
ax.semilogy(qubit_range, variances, 'o-', color='#FF6B6B', linewidth=2,
           markersize=10, label='Hardware-efficient + Global cost')

# fit exponential decay line for reference
log_vars = np.log(variances)
coeffs = np.polyfit(qubit_range, log_vars, 1)
fit_line = np.exp(np.polyval(coeffs, qubit_range))
ax.semilogy(qubit_range, fit_line, '--', color='gray', linewidth=1.5,
           label=f'Exponential fit (slope={coeffs[0]:.2f})', alpha=0.7)

ax.set_xlabel('Number of Qubits')
ax.set_ylabel('Gradient Variance')
ax.set_title('Barren Plateau: Gradient Variance vs Circuit Width')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'variance_vs_qubits.png', dpi=150)
plt.show()

print(f"Exponential decay rate: {coeffs[0]:.3f} per qubit")
print(f"This means each added qubit reduces variance by ~{np.exp(coeffs[0]):.2f}x")

## 3. Experiment 2: Global vs Local Cost Function

Cerezo et al. (2021) showed that the choice of cost function matters. **Global cost functions** (that measure all qubits) lead to barren plateaus even for shallow circuits, while **local cost functions** (that measure only a few qubits) can avoid them for logarithmic-depth circuits.

Let's test this.

In [ ]:
# same sweep but with local cost function
print("Sweeping with LOCAL cost function...")
hw_local_results = sweep_gradient_variance(
    qubit_range=qubit_range,
    depth=depth,
    ansatz_builder=build_hardware_efficient_ansatz,
    observable_builder=local_cost_observable,
    n_samples=n_samples
)

# compare
fig, ax = plt.subplots(figsize=(10, 6))

ax.semilogy(qubit_range, hw_global_results['variances'], 'o-', color='#FF6B6B',
           linewidth=2, markersize=8, label='Global cost (sum Z_i)')
ax.semilogy(qubit_range, hw_local_results['variances'], 's-', color='#4ECDC4',
           linewidth=2, markersize=8, label='Local cost (Z_0 only)')

ax.set_xlabel('Number of Qubits')
ax.set_ylabel('Gradient Variance')
ax.set_title('Global vs Local Cost Function (depth=4)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'global_vs_local.png', dpi=150)
plt.show()

print("\nComparison:")
for n, g_var, l_var in zip(qubit_range, hw_global_results['variances'],
                           hw_local_results['variances']):
    ratio = l_var / g_var if g_var > 0 else float('inf')
    print(f"  {n} qubits: global={g_var:.2e}, local={l_var:.2e}, ratio={ratio:.1f}x")

## 4. Experiment 3: Gradient Variance vs Circuit Depth

Deeper circuits are more expressive but potentially less trainable. Let's fix the number of qubits and sweep the depth.

In [ ]:
# sweep depth at fixed qubit count
fixed_qubits = 6
depth_range = [1, 2, 4, 8, 12, 16]

print(f"Sweeping depth {depth_range} at {fixed_qubits} qubits...")
depth_results = sweep_depth_variance(
    n_qubits=fixed_qubits,
    depth_range=depth_range,
    ansatz_builder=build_hardware_efficient_ansatz,
    observable_builder=global_cost_observable,
    n_samples=n_samples
)

fig, ax = plt.subplots(figsize=(10, 6))
ax.semilogy(depth_range, depth_results['variances'], 'o-', color='#45B7D1',
           linewidth=2, markersize=8)
ax.set_xlabel('Circuit Depth (layers)')
ax.set_ylabel('Gradient Variance')
ax.set_title(f'Gradient Variance vs Circuit Depth ({fixed_qubits} qubits, global cost)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'variance_vs_depth.png', dpi=150)
plt.show()

## 5. Experiment 4: Ansatz Architecture Comparison

Not all circuit architectures are equally prone to barren plateaus. We compare:

- **Hardware-efficient**: full Ry/Rz rotations + linear CNOT entanglement (standard, known to have BPs)
- **Structured**: limited entanglement, fewer parameters (designed to be more trainable)
- **Identity-block**: initialized near identity (Grant et al. 2019 strategy)

In [ ]:
# compare three ansatz architectures
ansatz_builders = get_ansatz_builders()
ansatz_results = {}

for name, builder in ansatz_builders.items():
    print(f"\nSweeping {name}...")
    results = sweep_gradient_variance(
        qubit_range=qubit_range,
        depth=4,
        ansatz_builder=builder,
        observable_builder=global_cost_observable,
        n_samples=n_samples
    )
    ansatz_results[name] = results

# plot comparison
fig, ax = plt.subplots(figsize=(10, 6))

colors_ansatz = {'Hardware-Efficient': '#FF6B6B', 'Structured': '#4ECDC4',
                 'Identity-Block': '#45B7D1'}
markers = {'Hardware-Efficient': 'o', 'Structured': 's', 'Identity-Block': '^'}

for name, results in ansatz_results.items():
    ax.semilogy(qubit_range, results['variances'],
               marker=markers.get(name, 'o'), color=colors_ansatz.get(name, 'gray'),
               linewidth=2, markersize=8, label=name)

ax.set_xlabel('Number of Qubits')
ax.set_ylabel('Gradient Variance')
ax.set_title('Ansatz Architecture Comparison (depth=4, global cost)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'ansatz_comparison.png', dpi=150)
plt.show()

## 6. Variance Heatmap

Let's build a 2D picture: gradient variance as a function of both width AND depth. This gives a complete map of the trainability landscape.

In [ ]:
# compute variance for a grid of (width, depth) pairs
width_values = [2, 4, 6, 8, 10]
depth_values = [1, 2, 4, 8, 12]

variance_matrix = np.zeros((len(depth_values), len(width_values)))

for i, d in enumerate(depth_values):
    for j, n in enumerate(width_values):
        print(f"  n={n}, depth={d}...", end=" ")
        circuit = build_hardware_efficient_ansatz(n, d)
        observable = global_cost_observable(n)
        var = compute_gradient_variance(
            circuit, observable, param_index=0, n_samples=100
        )
        variance_matrix[i, j] = var
        print(f"var={var:.2e}")

print("\nDone!")

In [ ]:
# heatmap
fig, ax = plt.subplots(figsize=(10, 7))

log_variance = np.log10(variance_matrix + 1e-10)
im = sns.heatmap(log_variance, annot=True, fmt='.1f',
                xticklabels=width_values, yticklabels=depth_values,
                cmap='RdYlGn_r', ax=ax, cbar_kws={'label': 'log10(Variance)'})

ax.set_xlabel('Number of Qubits')
ax.set_ylabel('Circuit Depth')
ax.set_title('Gradient Variance Heatmap (Hardware-Efficient, Global Cost)')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'variance_heatmap.png', dpi=150)
plt.show()

print("Green = trainable, Red = barren plateau")
print("The bottom-right corner (wide + deep) is the danger zone.")

## 7. Cost Landscape Visualization

To build more intuition, let's visualize 1D slices through the cost landscape. For a trainable circuit, we should see clear structure (hills and valleys). For a barren plateau, it should look flat.

In [ ]:
# 1D cost landscape slices for small vs large circuits
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (n_q, title) in enumerate([(2, '2 qubits (trainable)'),
                                     (10, '10 qubits (barren plateau)')]):
    circuit = build_hardware_efficient_ansatz(n_q, depth=4)
    observable = global_cost_observable(n_q)
    
    # fix all params except one, sweep that one from 0 to 2pi
    rng = np.random.default_rng(42)
    base_params = rng.uniform(0, 2 * np.pi, circuit.num_parameters)
    
    theta_range = np.linspace(0, 2 * np.pi, 100)
    costs = []
    
    from qiskit.quantum_info import Statevector, SparsePauliOp
    
    for theta in theta_range:
        params = base_params.copy()
        params[0] = theta
        
        bound_circuit = circuit.assign_parameters(params)
        sv = Statevector(bound_circuit)
        cost = sv.expectation_value(observable).real
        costs.append(cost)
    
    axes[idx].plot(theta_range, costs, linewidth=2, color='#2196F3')
    axes[idx].set_xlabel('θ₀')
    axes[idx].set_ylabel('⟨C⟩')
    axes[idx].set_title(title)
    axes[idx].grid(True, alpha=0.3)
    
    # show variance of this slice
    cost_var = np.var(costs)
    axes[idx].annotate(f'Cost variance: {cost_var:.4f}',
                      xy=(0.05, 0.95), xycoords='axes fraction',
                      fontsize=10, verticalalignment='top',
                      bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('1D Cost Landscape Slices', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'cost_landscape.png', dpi=150, bbox_inches='tight')
plt.show()
print("Notice how the 10-qubit landscape is much flatter — harder to optimize.")

## 8. Layer-wise Training Strategy

One proposed mitigation strategy is **layer-wise training**: instead of optimizing all parameters of a deep circuit at once (which hits the barren plateau), build the circuit one layer at a time:

1. Start with 1 layer, optimize those parameters
2. Freeze them, add a second layer, optimize the new parameters
3. Repeat until you reach the desired depth

This way, you never optimize in the full exponentially-flat landscape.

In [ ]:
# layer-wise training on 8 qubits
n_q = 8
max_layers = 6
observable = global_cost_observable(n_q)

print(f"Layer-wise training: {n_q} qubits, up to {max_layers} layers")
layer_histories = layerwise_train(
    ansatz_builder=build_hardware_efficient_ansatz,
    cost_observable=observable,
    n_qubits=n_q,
    max_layers=max_layers,
    maxiter=50,
    seed=42
)

# plot convergence for each layer
fig, ax = plt.subplots(figsize=(10, 6))

colors_lw = plt.cm.viridis(np.linspace(0.1, 0.9, max_layers))
offset = 0

for i, history in enumerate(layer_histories):
    iters = range(offset, offset + len(history))
    ax.plot(iters, history, color=colors_lw[i], linewidth=1.5,
            label=f'Layer {i+1}')
    ax.axvline(x=offset, color='gray', linestyle=':', alpha=0.3)
    offset += len(history)

ax.set_xlabel('Total Iterations')
ax.set_ylabel('Cost')
ax.set_title(f'Layer-wise Training Convergence ({n_q} qubits)')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'layerwise_convergence.png', dpi=150)
plt.show()

## Summary

### What we confirmed:

1. **Barren plateaus are real** — gradient variance decays exponentially with qubit count for hardware-efficient ansatze with global cost functions, exactly as predicted by McClean et al. (2018).

2. **Local cost functions help significantly** — switching from a global to local observable increases gradient variance by orders of magnitude, consistent with Cerezo et al. (2021). This is probably the single most impactful design choice.

3. **Circuit architecture matters** — structured ansatze with limited entanglement show slower variance decay than fully random hardware-efficient designs. Identity-block initialization also helps preserve gradient information.

4. **Layer-wise training works** — by building the circuit incrementally, we avoid the worst of the barren plateau problem. Each layer starts from a reasonable initial point.

5. **Width AND depth both contribute** — the heatmap shows that both wider and deeper circuits have worse trainability, but width has the stronger effect.

### Implications for QML algorithm design:

- Use **local cost functions** whenever possible
- Prefer **shallow circuits** with problem-aware structure over deep random circuits
- Use **layer-wise or incremental training** for deeper circuits
- **Initialize carefully** — random initialization in wide circuits almost guarantees you start in a barren plateau
- The barren plateau problem sets a practical limit on the scalability of current variational QML approaches